<a href="https://colab.research.google.com/github/alessandronascimento/pyLiBELa/blob/main/Colabs/pyLiBELa_SB2021_cuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First things first
### From now on, we will be only using CUDA, a GPU computing API.
### In order to do that, we need to use the GPU runtime from Google Colab that can be selected following the steps in the menu:
### `Runtime > Change Runtime Type > T4 GPU`

In [1]:
#@title Downloading dependencies {display-mode: "form"}

%%capture
! apt-get install python-dev-is-python3 zlib1g-dev libeigen3-dev libgsl-dev libnlopt-cxx-dev
!pip install ipython-autotime
%load_ext autotime
!pip3 install condacolab
import condacolab
condacolab.install()
!mamba install openbabel


time: 44.5 s (started: 2023-09-25 18:21:34 +00:00)


In [1]:
#@title Installing pyLiBELa {display-mode: "form"}

%%capture
! git clone --branch alex-works https://github.com/alessandronascimento/pyLiBELa.git
! mv pyLiBELa/src src
! rm -rf pyLiBELa
! mkdir -p obj
! rm -f Makefile*
! wget https://raw.githubusercontent.com/alessandronascimento/pyLiBELa/alex-works/Colabs/Makefile
! sed -i 's+-I/usr/include/openbabel3+-I/usr/local/include/openbabel3+g' Makefile
! make -j2

In [3]:
#@title Importing pyLiBELa {display-mode: "form"}
import random

try:
  from pyPARSER import *
  from pyMol2 import *
  from pyWRITER import *
  from pyGrid import *
  from pyCOORD_MC import *
  from pyFindHB import *
  from pyEnergy2 import *
  from pyGaussian import *
  from pyConformer import *
  from pyRAND import *
  from pyMcEntropy import *
  from pySA import *
  from pyOptimizer import *
  from pyMC import *
  from pyFullSearch import *
  from pyDocker import *
  print('pyLiBELa is imported!')
except ImportError:
  print('An ImportError occurred, try running this cell again!')

pyLiBELa is imported!


In [4]:
#@title Getting SB2021 data {display-mode: "form"}

%%capture
from google.colab import drive
drive.mount('/content/drive/')
sb_folder = '/content/drive/MyDrive/pyLiBELa/SB/' #@param {type:"string"}

%cd $sb_folder
targets=[]
targets_file = open('list', 'r');
for line in targets_file:
  targets.append(line.strip())
targets_file.close()


In [5]:
#@title Docking parameters {display-mode: "form"}

import os
import timeit
import numpy as np

Input = PARSER()

# Some default options:
# 1. We are using two processors for Grid calculations;
#

Input.generate_conformers = True;
Input.dock_parallel = True;
Input.parallel_jobs = 2;
Input.write_grids = True
Input.use_grids = True
Input.write_mol2 = True
Input.atom_limit = 150 #@param {type:"number"}
    #atom limit not counting H

# Energy Calculation Parameters:
scoring_function = "0" #@param ["0", "1", "2", "3"]
Input.dielectric_model = "r" #@param ["r", "constant"]
Input.scoring_function = int(scoring_function)
grid_dimension = 30.0 #@param {type:"number"}
Input.grid_spacing = 0.5 #@param {type:"number"}
Input.solvation_alpha = 0.1 #@param {type:"number"}
Input.solvation_beta = -0.005 #@param {type:"number"}

# Optimization parameter:
Input.min_tol = 1E-4;
Input.min_delta = 1E-5;
Input.dock_min_tol = 1E-4;
search_box = 30.0 #@param {type:"number"}
Input.timeout = 30 #@param {type:"number"}
Input.min_timeout = 30 #@param {type:"number"}
Input.overlay_optimizer = "mma" #@param ["mma", "ln_auglag", "subplex", "none"]
Input.energy_optimizer = "mma" #@param ["direct", "isres", "crs", "esch", "stogo", "mma", "simplex", "none"]
if (Input.scoring_function < 3):
  delta = 2.5 #@param {type:"number"}
  Input.deltaij6 = (delta*delta*delta*delta*delta*delta)
  delta_es = 2.5 #@param {type:"number"}
  Input.deltaij_es6 = pow(delta_es, 6);
  Input.deltaij_es3 = (delta_es*delta_es*delta_es)

Input.search_box_x, Input.search_box_y, Input.search_box_z = search_box, search_box, search_box;
Input.x_dim, Input.y_dim, Input.z_dim = grid_dimension, grid_dimension, grid_dimension;
Input.grid_prefix = "McGrid"

In [8]:
#@title Docking {display-mode: "form"}

%load_ext autotime



print('#%10s %10s' % ('TARGET', 'RMSD(Ang)'))

targets_random=random.sample(targets, len(targets));


target_cut = targets[:5]

for target in target_cut:
#  target=targets_random[68]
  folder = sb_folder+target
#  print()
#  print()
  %cd $folder
  ! mkdir -p test
  %cd test
  ! rm -f *

  print()
  print()

  ! cp ../*lig.am1bcc.mol2.gz lig.mol2.gz
  ! cp ../*.rec.clean.mol2.gz rec.mol2.gz
  ! obabel -imol2 lig.mol2.gz -osmi -O lig2.smi &> /dev/null #self docking

  insmiles = open('lig2.smi', 'r');
  line=insmiles.readline().strip()
  line2 = line.split()
  ligand_smiles = line2[0]
  insmiles.close();

  Lig2 = Mol2()
  ligcheck = Lig2.parse_smiles(Input, ligand_smiles, 'LIG')
#  if (not ligcheck):
#    print("#Ops! A problem was identified in ligand %s" % target);

  lig_src =  'lig.mol2.gz'
  rec_src = 'rec.mol2.gz'

  REC = Mol2(Input, rec_src)
  RefLig = Mol2(Input, lig_src)

#  print("Receptor has %4d atoms." % REC.N)
#  print('Reference Ligand has %4d atoms' % RefLig.N)
#  print('Search ligand has %4d atoms' % Lig2.N)

  if (ligcheck):
    Writer = WRITER(Input)
    Coord = COORD_MC()
    HB = FindHB()
    Ene  = Energy2(Input)

    for i in range(len(REC.residue_pointer)-1):
      HB.parse_residue(REC.residue_pointer[i]-1, REC.residue_pointer[i+1]-2, REC.resnames[i], REC, RefLig, 9.0);
    HB.find_ligandHB(lig_src, RefLig);

    print('The receptor has %5d / %5d HB donors/acceptors around the active site.' % (len(REC.HBdonors), len(REC.HBacceptors)));

    Dock = Docker(Writer)
    center = Coord.compute_com(RefLig)

#    print()

    start_energy = Ene.compute_ene(REC, RefLig, RefLig.xyz);

    print('Starting energy: %7.3f kcal/mol' % start_energy);
    print('Generating grids. This may take a while..')


    start_time = timeit.default_timer()
    Grids = Grid(Input, Writer, REC, center)
    time_grid = timeit.default_timer() - start_time

    time_per_atom = time/REC.N
    print('Grids computed, and it took %.2f s! That means %.4f s per atom.' %(time,time_per_atom))

    grid_energy = Ene.compute_ene(Grids, RefLig, RefLig.xyz);

    print('Grid original energy: %7.3f kcal/mol' % grid_energy);
    print('Grid error: %7.3f %%' % abs(100.*(start_energy-grid_energy)/start_energy));

    print('Starting docking calculation...')

    start_time = timeit.default_timer()
    Dock.run(REC, Lig2, RefLig, center, Input, Grids, 0)
    time_dock = timeit.default_timer() - start_time

#    Computing the RMSD

    ! obabel -imol2 McLiBELa_dock.mol2.gz -pdb -O docked_ligand.pdb &> /dev/null
    rmsd = ! obrms lig.mol2.gz docked_ligand.pdb
    if (len(rmsd) > 0):
      rmsd = rmsd[0].split()
#      print(rmsd)
      if (len(rmsd) == 3):
        print('%10s %10.10s Ang' % (target, rmsd[2]))
    else:
      print('%10s %10.10s Ang' % (target, 'N/A'));
    print('***********************************************************************************************')


#    TARGET  RMSD(Ang)
/content/drive/MyDrive/pyLiBELa/SB/121P
/content/drive/MyDrive/pyLiBELa/SB/121P/test


The receptor has    24 /    25 HB donors/acceptors around the active site.
Starting energy: -144.121 kcal/mol
Generating grids. This may take a while..
Grids computed, and it took 3.36 s! That means 0.0013 s per atom.
Grid original energy:  26.705 kcal/mol
Grid error: 118.530 %
Starting docking calculation...
      121P        N/A Ang
***********************************************************************************************
/content/drive/MyDrive/pyLiBELa/SB/181L
/content/drive/MyDrive/pyLiBELa/SB/181L/test
rm: cannot remove 'obj': Is a directory
rm: cannot remove 'src': Is a directory
rm: cannot remove 'test': Is a directory


The receptor has    38 /    26 HB donors/acceptors around the active site.
Starting energy: -12.525 kcal/mol
Generating grids. This may take a while..
Grids computed, and it took 3.38 s! That means 0.0013 s per atom.
Grid original energy: -12.427 kc